In [23]:
from py_ecc.bn128 import G1, multiply, add, FQ, eq, Z1, curve_order
from py_ecc.bn128 import curve_order as p
import numpy as np
from functools import reduce
import random
import galois
import sys
sys.path.append('../hw_1')
from pedersen_commitment import generate_n_random_points, pedersen_commitment

In [24]:
GF = galois.GF(curve_order)

In [25]:
def gf_inverse(a):
    return GF(1) / a

In [26]:
def random_element():
    return random.randint(0, 1000)

def add_points(points):
    return reduce(add, points, Z1)

# if points = G1, G2, G3, G4 and scalars = a,b,c,d vector_commit returns
# aG1 + bG2 + cG3 + dG4
def vector_commit(points, scalars):
    # return reduce(add, [multiply(P, i) for P, i in zip(points, scalars)], Z1)
    result = Z1
    for P, i in zip(points, scalars):
        result = add(result, multiply(P, int(i)))
    return result


In [27]:
G_vec = generate_n_random_points('hello', 2)

In [28]:
# return a folded vector of length n/2 for scalars
def fold(scalar_vec, u):
    # if scalar_vec is odd add 0 to the end
    if len(scalar_vec) % 2 == 1:
        scalar_vec.append(0)
    # fold the vector
    return [scalar_vec[i] * u + gf_inverse(u) * scalar_vec[i + 1] for i in range(0, len(scalar_vec), 2)]


In [29]:
# return a folded vector of length n/2 for points
def fold_points(point_vec, u):
    if len(point_vec) % 2 == 1:
        point_vec.append(Z1)
    # fold the vector
    return [add(multiply(point_vec[i], int(u)), multiply(point_vec[i + 1], int(gf_inverse(u)))) for i in range(0, len(point_vec), 2)]


In [30]:
points_vec = generate_n_random_points('hello', 4)
print(len(fold_points(points_vec, GF(2))))


2


In [31]:
def compute_secondary_diagonal(G_vec, a):
    if len(a) % 2 == 1:
        a.append(0)
        G_vec.append(Z1)
    
    L = vector_commit([G_vec[i+1] for i in range(0, len(G_vec)-1, 2)], [a[i] for i in range(0, len(a)-1, 2)])
    R = vector_commit([G_vec[i] for i in range(0, len(G_vec)-1, 2)], [a[i+1] for i in range(0, len(a)-1, 2)])
    
    return L, R

In [32]:
def verify(A, L, R, a_prime, u, G_vec):
    # Calculate right hand side: L·u² + A + R·u⁻²
    u_squared = int((u * u))
    u_inv_squared = int((gf_inverse(u) * gf_inverse(u)))
    
    rhs = multiply(L, u_squared)  # Start with L·u²
    rhs = add(rhs, A)            # Add A
    rhs = add(rhs, multiply(R, u_inv_squared))  # Add R·u⁻²

    # Calculate left hand side
    # <folded_g, a_prime>
    folded_G = fold_points(G_vec, gf_inverse(u))
    lhs = vector_commit(folded_G, a_prime)

    return eq(lhs, rhs)

In [33]:
a = [9,45,23,42]

# prover commits
A = vector_commit(G_vec, a)
L, R = compute_secondary_diagonal(G_vec, a)

# verifier computes randomness
u = GF(random_element())

# prover computes fold(a)
aprime = fold(a, u)

# verifier computes fold(G)
# Gprime = fold_points(G_vec, GF(pow(u, -1)))

print(verify(A, L, R, aprime, u, G_vec))

# # verification check
# assert eq(vector_commit(Gprime, aprime), add_points(multiply(L, pow(u, 2, p)), A, multiply(R, pow(u, -2, p)))), "invalid proof"
# assert len(Gprime) == len(a) // 2 and len(aprime) == len(a) // 2, "proof must be size n/2"

False
